#**Entrenamiento del modelo en español Word2Vec**
Primero se investigó acerca de modelos en español, entrenados para reconocimiento y segmentación de palabras (tokenización) en español. Uno de los más completos fue "cc.es.300.vec.gz". Disponible en: https://fasttext.cc/docs/en/crawl-vectors.html

Distribuimos vectores de palabras previamente entrenados para 157 idiomas, entrenados en Common Crawl y Wikipedia usando fastText. Estos modelos fueron entrenados utilizando CBOW con posición-pesos, en dimensión 300, con caracteres n-gramas de longitud 5, ventana de tamaño 5 y 10 negativos. También distribuimos tres nuevos conjuntos de datos de analogía de palabras, para francés, hindi y polaco. Puede encontrar más información sobre el entrenamiento de estos modelos en el artículo Aprendiendo vectores de palabras para 157 idiomas.

**Licencia:** Creative Commons Attribution-Share-Alike License 3.0.

**Referencias**: El artículo referenciado es: [E. Grave*, P. Bojanowski*, P. Gupta, A. Joulin, T. Mikolov, Learning Word Vectors for 157 Languages](https://arxiv.org/abs/1802.06893)

Otros recursos de código: https://github.com/pushpendudas/custom_word2vec_model_training/blob/master/word2vec%20training.ipynb

In [ ]:
#INSTALAR LIBRERIAS
!pip install PyPDF2

     |████████████████████████████████| 77 kB 2.8 MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61101 sha256=a4207ab14285bf4aca812669b3886b63e2a488c1deceb0ee6673e106ad4c08ec
  Stored in directory: /root/.cache/pip/wheels/80/1a/24/648467ade3a77ed20f35cfd2badd32134e96dd25ca811e64b3
Successfully built PyPDF2


In [ ]:
#Medición de las Dimensión socio-cultural: aparte del ODS,trasnversal ala agenda unsco 2030
#https://lab.ccesv.org/los-derechos-culturales-como-parte-de-la-agenda-2030-es-posible-su-cumplimiento-tras-la-pandemia/

In [ ]:
#IMPORTAR LIBRERIAS
import os, sys
import seaborn as sns
#import PyPDF2
from nltk import Tree
from spacy import displacy
from spacy.pipeline import DependencyParser
import re
import spacy
from spacy.lang.es import Spanish
from spacy.pipeline import EntityRuler
import nltk
from nltk.corpus import stopwords
from nltk.data import load
from string import punctuation
nltk.download('stopwords')
spanish_stopwords=stopwords.words('spanish')
non_words=list(punctuation)
import sys
sys.path.append("..")
from itertools import chain
import more_itertools as mit
from nltk.corpus import names 
from nltk.corpus.reader.util import *
import nltk 
from nltk import precision
import collections
import nltk.metrics
from nltk.tokenize import MWETokenizer
nltk.download('names')
import random 
from nltk.corpus import names 
import nltk


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.


In [ ]:
import time
from tqdm import tqdm, trange, tqdm_notebook
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#IMPORTAR EL MODELO DESDE EL DRIVE, TAMBIEN SE PUEDEN SEGUIR LAS INSTRUCCIONES DE 
#https://fasttext.cc/docs/en/crawl-vectors.html
downloaded = drive.CreateFile({'id':"XXX"})   # replace the id with id of file you want to access
downloaded.GetContentFile('cc.es.300.vec.gz') 

In [ ]:
downloaded = drive.CreateFile({'id':"XXX"})   # replace the id with id of file you want to access
downloaded.GetContentFile('ODS (1).pdf') 
downloaded = drive.CreateFile({'id':"XXX"})   # replace the id with id of file you want to access
downloaded.GetContentFile('ODS (2).pdf') 
downloaded = drive.CreateFile({'id':"XXX"})   # replace the id with id of file you want to access
downloaded.GetContentFile('ODS (3).pdf') 
downloaded = drive.CreateFile({'id':"XXX"})   # replace the id with id of file you want to access
downloaded.GetContentFile('ODS (4).pdf') 
downloaded = drive.CreateFile({'id':"XX"})   # replace the id with id of file you want to access
downloaded.GetContentFile('Lineas_Concytec.pdf')
downloaded = drive.CreateFile({'id':"XX"})   # replace the id with id of file you want to access
downloaded.GetContentFile('NCUBE_LINES.pdf')  

In [ ]:
#ABRIR EL MODELO
import gzip 
import shutil 
with gzip.open('cc.es.300.vec.gz', 'rb') as f_in: 
  with open('cc.es.300.vec', 'wb') as f_out: 
    shutil.copyfileobj(f_in, f_out) 

In [ ]:
from gensim.models.keyedvectors import KeyedVectors
#LEER EL MODELO EN ESPAÑOL DE WORD2VEC
wordvectors_file_vec = 'cc.es.300.vec'#wiki.es.vec'#'cc.es.300.vec'
cantidad = 100000
wordvectors = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)

In [ ]:
#PROBAR EL MODELO EN BLANCO
wordvectors.most_similar_cosmul('patrimonial')

[('patrimonio', 0.8957945108413696),
 ('patrimoniales', 0.865646481513977),
 ('Patrimonial', 0.8525913953781128),
 ('patrimonios', 0.8207882046699524),
 ('cultural', 0.8054655194282532),
 ('Patrimonio', 0.7939102649688721),
 ('histórico-artístico', 0.7744946479797363),
 ('arquitectónico', 0.7725043296813965),
 ('paisajístico', 0.7723819017410278),
 ('jurídico', 0.7638840079307556)]

In [ ]:
wordvectors.similarity('sostenibilidad', 'ambiental')

0.5924913

In [ ]:
#LISTAR LOS DOCUMENTOS OFICIALES
nombres=['ODS (1).pdf',
        'ODS (2).pdf',
        'ODS (3).pdf',
        'ODS (4).pdf']

In [ ]:
#LEER TODOS LOS PDF
i=0
for archivo in nombre:
  i=i+1
  exec(f'pdfFileObj{i} = open(nombre[i], "rb")')
  exec(f'pdfReader{i} = PyPDF2.PdfFileReader(pdfFileObj{i})')
  exec(f'Pages=pdfReader{i}.numPages')## LEER PDF DE LEY Y CONVERTIR A TEXTO PLANO
  #Almacenar el contenido por paginas
  Caps=[]
  for k in trange(0,Pages):
      exec(f'pageObj = pdfReader{i}.getPage(k)')
      Cap=pageObj.extractText()#.encode('utf8')
      Caps.append(Cap)
      XX={'Texto'}
  exec(f'df{i}=pd.DataFrame(Caps,columns=XX)')
  exec(f'df{i}["Archivo"]=nombres[i]')

In [ ]:
i=1#,2,3,4
df=pd.DataFrame(Caps,columns={'Texto'})
df['Archivo']=nombre[i]
df1=pd.DataFrame(Caps,columns={'Texto'})
df1['Archivo']=nombre[i]
df2=pd.DataFrame(Caps,columns={'Texto'})
df2['Archivo']=nombre[i]
df3=pd.DataFrame(Caps,columns={'Texto'})
df3['Archivo']=nombre[i]
df4=pd.DataFrame(Caps,columns={'Texto'})
df4['Archivo']=nombre[i]

In [ ]:
db=[df,df1,df2,df3,df4]
result=pd.concat(db,ignore_index=True)
len(result)

In [ ]:
#PREPROCESO DE TEXTO
def preprocess(text):
    nlp = Spanish()
    result= []
    for i in trange(0,len(text)):
        text[i]=str(text[i])
        text[i]=re.sub(u'^(?<![a-z A-Z])',' ',text[i])
        text[i]=text[i].lower()
        text[i]=text[i].replace('"',' ')
        text[i]=text[i].replace('(',' ')
        text[i]=text[i].replace(')',' ')
        text[i]=text[i].replace('\n',' ')
        text[i]=text[i].replace(']',' ')
        text[i]=text[i].replace('[',' ')
        text[i]=text[i].replace('/',' ')
        text[i]=text[i].replace('%',' ')
        text[i]=text[i].replace('+',' ')
        text[i]=text[i].replace('-',' ')
        text[i]=text[i].replace('.',' ')
        text[i]=text[i].replace(';',' ')
        text[i]=text[i].replace('\n',' ')
        text[i]=text[i].replace(':',' ')
        text[i]=text[i].replace('|',' ')
        text[i]=text[i].replace('*',' ')
        text[i]=text[i].replace('—',' ')
        text[i]=text[i].replace('€',' ')
        text[i]=text[i].replace('\\n',' ')
        text[i]=text[i].replace(',','')
        text[i]=text[i].replace(' ma ','')
        text[i]=text[i].replace(' aqui ','')
        text[i]=text[i].replace('cuales ','')
        text[i]=text[i].replace('toda ','')
        text[i]=text[i].replace('cada ','')
        text[i]=text[i].replace(' lo ','')
        a,b = 'áéíóúü','aeiouu'
        trans = str.maketrans(a,b)
        text[i]=text[i].translate(trans)
        text[i]=text[i].strip()
        doc=nlp(text[i])
        t=[token.text for token in doc]
        result.append(' '.join(list(t)))
    return result

In [ ]:
result['Prep_text']=preprocess(result['Texto'])
result['Prep_text'][8]

In [ ]:
# Loading libraries
import re
import time
import pandas as pd
from tqdm.auto import tqdm
from collections import Counter

from nltk import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from gensim.models import Word2Vec

In [ ]:
# Time calculation
def cal_elapsed_time(s):
    return print("Elapsed time:\t", round((time.time() - s),2))
s_time = time.time()
cal_elapsed_time(s=s_time)

Elapsed time:	 0.0


In [ ]:
#OBTENER LA INFORMACION
data_df=result#.head()

In [ ]:
#Model training
#Using Gensim model to triain word2vec model
from gensim.models import Word2Vec

In [ ]:
s_time = time.time()
print("Model Training Started...")
####wordvectors = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)
#### model =wordvectors(sentences=sent_corpus, size=200, window=4, min_count=1, workers=4)
#model = Word2Vec(sentences=sent_corpus, size=200, window=4, min_count=1, workers=4)
cal_elapsed_time(s_time)

In [ ]:
print("Total number of unique words loaded in Model : ", len(model.wv.vocab))#3300

In [ ]:
#GUARDAR EL MODELO
model.save("data/model/trainned_model.model")

In [ ]:
# Find similar words for efficiency
model.wv.most_similar("agua", topn=20)

[('sostenible', 0.8844438195228577),
 ('persona', 0.8756629228591919),
 ('desarrollo', 0.8702710270881653),
 ('recursos', 0.8692913055419922),
 ('mujeres', 0.8688398599624634),
 ('dea', 0.8618358373641968),
 ('proporcion', 0.8614614009857178),
 ('paises', 0.8496870994567871),
 ('nacionales', 0.8466477990150452),
 ('ma', 0.8452156186103821),
 ('servicios', 0.844576358795166),
 ('toda', 0.8400709629058838),
 ('numero', 0.8389781713485718),
 ('sostenibles', 0.838219165802002),
 ('mundial', 0.8336294293403625),
 ('agenda', 0.8332079648971558),
 ('acceso', 0.8326206207275391),
 ('promover', 0.832047700881958),
 ('trabajo', 0.8317737579345703),
 ('consumo', 0.8223632574081421)]

In [ ]:
# List of word similarity
model.wv.n_similarity(['igualdad','mujer'],['genero','mujeres'])

0.6695497

In [ ]:
# Distance between two words
model.wv.distance('igualdad','mujer')

0.8909858763217926

IMPORTANTE: Para conocer las palabras similares en los dicionarios, se construyen segun los temas por ejemplo: Para DSA: ambiente o ambiental o lo relacionado con medios urbanos se lista